In [ ]:
import time 
import tqdm
import torch
import torch.nn as nn  
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.utils import make_grid

from IPython.display import HTML
%matplotlib inline

# Using GPU
Before executing the cell, go to Runtime -> Change Runtime Type -> GPU

In [ ]:
X = torch.randn(3, 2)
print(X)

In [ ]:
X.device

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
X = torch.randn(3, 2)
print(X.device)

# Prepare MNIST dataset

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = '/content/gdrive/MyDrive/' # path to save MNIST dataset

In [ ]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))
])
dataset = MNIST(path, train=True, download=True,
                transform = transform)           


In [ ]:
len(dataset)

In [ ]:
# Show a random image
idx = np.random.choice(len(dataset))
img, label = dataset[idx]
print('Image size: {}'.format(img.shape))
print('Label: {}'.format(label))
plt.axis('off')
plt.imshow(img.permute(1, 2, 0).squeeze(), cmap='gray')
plt.show()

In [ ]:
def show_images(image_tensor, num_images=25, nrow=5, save=False): 
  image_tensor = image_tensor.detach().to('cpu') 
  image_tensor = (image_tensor + 1)/2  
  img = make_grid(image_tensor[:num_images], nrow=nrow).permute(1,2,0).squeeze()  
  if save:
    return img
  plt.axis('off')
  plt.imshow(img)
  plt.show()
  if save:
    return img

In [ ]:
# create a dataloader
batch_size = 128
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
for i, data in enumerate(dataloader):
  X, _ = data  
  break

show_images(X, 100, 10)

# Input noise vector

In [ ]:
def noise_vector(num, dim):
  # return a noise vector with width=height=1, channel=dim
  return torch.randn(num, dim, 1, 1)

In [ ]:
z = noise_vector(10, 100)
print(z.shape)

# Generator Model

**Each layer of generator:**


*   Transposed convolution for upsampling
*   Use batchnorm except for the last layer
*   Apply ReLU activation for all layers except for the output, which uses TanH

Remember that the output size of transposed convolution is:
$$output size = (input size -1)*stride - 2*padding + kernel size$$

Useful functions in building generator:


*   [ConvTranspose2d](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html)
*   [BatchNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html?highlight=batchnorm#torch.nn.BatchNorm2d)
*   [ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html?highlight=relu#torch.nn.ReLU)
*   [Tanh](https://pytorch.org/docs/stable/generated/torch.nn.Tanh.html?highlight=tanh#torch.nn.Tanh)






In [ ]:
class Generator(nn.Module):
  '''
  z_dim: the length of the input noise vector, a scalar
  hidden_dim: size of the feature maps that are propagated through the generator, a scalar
  out_channel: number of channels in the output image, set to 1 for MNIST (black and white)
  '''
  def __init__(self, zdim, hidden_dim=64, out_channel=1):
    super(Generator, self).__init__()
    self.model = nn.Sequential(
        # layer 1, input is z (noise)
        nn.ConvTranspose2d(zdim, hidden_dim*4, kernel_size=4),
        nn.BatchNorm2d(hidden_dim*4),
        nn.ReLU(),
        # size (hidden_dim*4) x 4 x 4

        # layer 2
        nn.ConvTranspose2d(hidden_dim*4, hidden_dim*2, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(hidden_dim*2),
        nn.ReLU(),
        # size (hidden_dim*2) x 8 x 8

        # layer 3
        nn.ConvTranspose2d(hidden_dim*2, hidden_dim, kernel_size=4, stride=2, padding=2),   
        nn.BatchNorm2d(hidden_dim),
        nn.ReLU(),           
        # size (hidden_dim) x 14 x 14

        # layer 4 (last layer)
        nn.ConvTranspose2d(hidden_dim, out_channel, kernel_size=4, stride=2, padding=1),
        nn.Tanh()
        # output size out_channel x 28 x 28
    )

  def forward(self, X):
    return self.model(X)

From the DCGAN paper, the author suggested to initialize all weights from a zero-centered Normal distribution with standard deviation 0.02.

In [ ]:
# apply to Generator and Discriminator network
def init_weights(m):
  if type(m) == nn.Conv2d or type(m) == nn.ConvTranspose2d:
    nn.init.normal_(m.weight, mean=0.0, std=0.02)
  elif type(m) == nn.BatchNorm2d:
    nn.init.normal_(m.weight, mean=0.0, std=0.02)
    nn.init.constant_(m.bias, val=0.0)

In [ ]:
gen = Generator(100)
gen.apply(init_weights)
print(gen)

In [ ]:
z = noise_vector(1, 100)
fake = gen(z).detach()
print(fake[0].shape)

plt.axis('off')
plt.imshow(fake[0].permute(1, 2, 0).squeeze(), cmap='gray')
plt.show()

# Discriminator Model

**Each layer of discriminator:**


*   Convolution for downsampling
*   Use batchnorm except for the last layer
*   Apply LeakyReLU activation with slope of 0.2 for all layers 

Remember that the output size of convolution is:
$$output size = (inputsize + 2*padding - kernelsize)/stride + 1$$

Useful functions in building discriminator:


*   [Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html?highlight=conv2d#torch.nn.Conv2d)
*   [BatchNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html?highlight=batchnorm#torch.nn.BatchNorm2d)
*   [LeakyReLU](https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html?highlight=leaky%20relu#torch.nn.LeakyReLU)
*   [Sigmoid](https://pytorch.org/docs/stable/nn.functional.html?highlight=sigmoid#torch.nn.functional.sigmoid)


In [ ]:
class Discriminator(nn.Module):
  '''
  im_channel: number of channels in the input image. Default is 1 for MNIST dataset
  hideen_dim: size of the feature maps that are propagated through the discriminator, a scalar
  '''
  def __init__(self, im_channel=1, hidden_dim=64):
    super(Discriminator, self).__init__()
    self.model = nn.Sequential(
        # layer 1, input is image of size im_channel x 28 x 28
        nn.Conv2d(im_channel, hidden_dim, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(hidden_dim),
        nn.LeakyReLU(0.02),
        # size (hidden_dim) x 14 x 14

        # layer 2
        nn.Conv2d(hidden_dim, hidden_dim*2, kernel_size=4, stride=2, padding=2),
        nn.BatchNorm2d(hidden_dim*2),
        nn.LeakyReLU(0.02),
        # size (hidden_dim*2) x 8 x 8

        # layer 3
        nn.Conv2d(hidden_dim*2, hidden_dim*4, kernel_size=4, stride=2, padding=1),
        nn.BatchNorm2d(hidden_dim*4),
        nn.LeakyReLU(0.02),
        # size (hidden_dim*4) x 4 x 4

        # layer 4 (last layer)
        nn.Conv2d(hidden_dim*4, 1, kernel_size=4),
        nn.Sigmoid()
    )
  
  def forward(self, image):
    out = self.model(image)
    return out.view(len(image), -1)

In [ ]:
disc = Discriminator()
disc.apply(init_weights)
print(disc)

# Start Training

In [ ]:
z_dim = 100
learning_rate = 0.0002
beta1 = 0.5
beta2 = 0.999

device = "cuda" if torch.cuda.is_available() else "cpu"

# loss function
criterion = nn.BCELoss()

# initialize generator, discriminator, optimizer
# optimizers for both Generator and Discriminator
gen = Generator(z_dim).to(device)
disc = Discriminator().to(device)


gen.apply(init_weights)
disc.apply(init_weights)

optimG = torch.optim.Adam(gen.parameters(), lr=learning_rate, betas=(beta1, beta2))
optimD = torch.optim.Adam(disc.parameters(), lr=learning_rate, betas=(beta1, beta2))

In [ ]:
# fixed noise for visualization purpose
fixed_noise = noise_vector(100, z_dim).to(device)

In [ ]:
num_epochs = 20

img_list = []
gen_losses = []
disc_losses = []

iter = 0

start_time = time.time()
for epoch in range(num_epochs):
  for i, data in enumerate(dataloader):
    real = data[0].to(device)

    # update discriminator
    optimD.zero_grad()
    noise = noise_vector(len(real), 100).to(device)
    fake = gen(noise).detach()
    disc_fake_pred = disc(fake)
    disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
    disc_real_pred = disc(real)
    disc_real_pred = disc(real)
    disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
    disc_loss = (disc_fake_loss + disc_real_loss)/2

    # record discrimator loss for later visualization purpose
    disc_losses.append(disc_loss.item())
    # calculate gradients of discriminator
    disc_loss.backward()
    # update optimizer
    optimD.step()
    
    # update generator
    optimG.zero_grad()
    noise2 = noise_vector(len(real), 100).to(device)
    fake2 = gen(noise2)
    disc_fake_pred2 = disc(fake2)
    gen_loss = criterion(disc_fake_pred2, torch.ones_like(disc_fake_pred2))

    # record generator loss for later visualization purpose
    gen_losses.append(gen_loss.item())
    # calculate gradients of generator
    gen_loss.backward()
    # updatde optimizer
    optimG.step()
       
    if iter % 500 == 0 or ((epoch == num_epochs-1) and (i == len(dataloader) - 1)):
      with torch.no_grad():
        fixed_fake = gen(fixed_noise)
        img = show_images(fixed_fake, len(fixed_fake), nrow=10, save=True)        
        img_list.append(img)
    iter+=1
  # training status
  print('[{}/{}]\tLoss_D: {:.5f}\tLoss_G: {:.5f}\tD(x): {:.5f}\tD(G(z)): {:.5f}/{:.5f}'.format(epoch+1, num_epochs, disc_loss.item(), gen_loss.item(), disc_real_pred.mean().item(), disc_fake_pred.mean().item(), disc_fake_pred2.mean().item()))

end_time = time.time()
print('Training process done! Time used: {} mins.'.format((end_time - start_time)/60))


# Visualization

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(gen_losses,label="G")
plt.plot(disc_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
fig = plt.figure()
plt.axis("off")
ims = [[plt.imshow(i, animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=500, repeat_delay=500, blit=True)

HTML(ani.to_html5_video())

In [3]:
% cd /content/gdrive/MyDrive/Github

/content/gdrive/MyDrive/Github


In [4]:
!git init dcgan

Initialized empty Git repository in /content/gdrive/MyDrive/Github/dcgan/.git/


In [7]:
!git status

On branch master

No commits yet

nothing to commit (create/copy files and use "git add" to track)
